<a href="https://colab.research.google.com/github/benbaz-2/comp551/blob/main/assignment4-merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

In [1]:
# Install packages
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# imports
from datasets import load_dataset
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModel
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

In [3]:
# Load GoEmotions dataset
dataset = load_dataset("google-research-datasets/go_emotions")

print(dataset['train'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'id'],
    num_rows: 43410
})


In [4]:
def filter_single_label(example):
    return len(example['labels']) == 1

# Apply filtering
train_data = dataset['train'].filter(filter_single_label)
val_data = dataset['validation'].filter(filter_single_label)
test_data = dataset['test'].filter(filter_single_label)


Filter:   0%|          | 0/43410 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5426 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5427 [00:00<?, ? examples/s]

For Baseline Models (SR, RF, XGBoost):Use TfidfVectorizer from scikit-learn

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['text'])
X_val = vectorizer.transform(val_data['text'])
X_test = vectorizer.transform(test_data['text'])

y_train = np.ravel(train_data['labels'])
y_val = np.ravel(val_data['labels'])
y_test = np.ravel(test_data['labels'])

For LLM (HuggingFace Transformers): Use a tokenizer from the transformers library

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_encodings = train_data.map(tokenize_data, batched=True)
val_encodings = val_data.map(tokenize_data, batched=True)
test_encodings = test_data.map(tokenize_data, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/36308 [00:00<?, ? examples/s]

Map:   0%|          | 0/4548 [00:00<?, ? examples/s]

Map:   0%|          | 0/4590 [00:00<?, ? examples/s]

In [7]:
print(train_encodings)

Dataset({
    features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36308
})


# Random Forest Classifier

In [8]:
# Example: Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbose=1, max_depth=100)
clf.fit(X_train, y_train)

# Validate and Test
y_pred = clf.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   15.9s


Test Accuracy: 0.5409586056644881


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   31.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


# Large Language Model

In [9]:
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_data['labels'])
)

val_dataset = TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_data['labels'])
)

test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_data['labels'])
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [10]:
# Define the path to save the model
save_path = "./pretrained_model"

# Download and save the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir=save_path)
base_model = AutoModel.from_pretrained("bert-base-uncased", cache_dir=save_path)

# Verify the save path
print(f"Model and tokenizer saved at: {save_path}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Model and tokenizer saved at: ./pretrained_model


In [11]:

class CustomBertModel(nn.Module):
    def __init__(self, base_model, num_labels):
        super().__init__()
        self.bert = base_model
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, label=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if label is not None:
            criterion = nn.CrossEntropyLoss()
            loss = criterion(logits, label)

        return {'loss':loss, "logits":logits} if loss is not None else {'logits':logits}
    def save_pretrained(self, save_directory):
        # Save tokenizer and model to the specified directory
        torch.save(self.state_dict(), f"{save_directory}/pytorch_model.bin")

In [12]:
num_labels =  28
model = CustomBertModel(base_model,num_labels)

In [14]:
# Define hyperparameters
learning_rate = 5e-5
epochs = 3
batch_size = 8
# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
# Total training steps (for learning rate scheduler)
num_training_steps = len(train_loader) * epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
# Loss function
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):

    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    train_loss=0
    for batch in tqdm(train_loader,desc='training'):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids,attention_mask=attention_mask)
        logits = outputs['logits']
        labels = labels.squeeze(-1)
        # compute loss
        loss = criterion(logits, labels)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Training loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validation'):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            # Forward
            outputs = model(input_ids = input_ids, attention_mask = attention_mask)
            logits = outputs["logits"]
            labels = labels.squeeze(-1)
            # Compute loss
            loss = criterion(logits, labels)
            val_loss += loss.item()

            # Compute accuracy
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    print(f"Validation loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}")


Epoch 1/3


training: 100%|██████████| 2270/2270 [12:25<00:00,  3.04it/s]


Training loss: 1.5353


Validation: 100%|██████████| 285/285 [00:32<00:00,  8.83it/s]


Validation loss: 1.2811, Accuracy: 0.6220
Epoch 2/3


...

Validation loss: 1.2319, Accuracy: 0.6313
Epoch 3/3


training: 100%|██████████| 2270/2270 [12:30<00:00,  3.02it/s]


Training loss: 0.7049


Validation: 100%|██████████| 285/285 [00:31<00:00,  8.91it/s]

Validation loss: 1.3368, Accuracy: 0.6146


In [23]:
#import os

# Define the directory name
#directory_name = "bert"

# Create the directory
#if not os.path.exists(directory_name):
    os.makedirs(directory_name)
#    print(f"Directory '{directory_name}' created.")
#else:
#    print(f"Directory '{directory_name}' already exists.")

# Save the model and tokenizer
#model.save_pretrained("bert")
#tokenizer.save_pretrained("bert")

#from transformers import pipeline

# Load fine-tuned model
#emotion_classifier = pipeline("text-classification", model="bert", tokenizer="bert")

# Test on custom text
#text = "I'm so happy to see you!"
#predictions = emotion_classifier(text)
#print(predictions)

#model.save_pretrained("./custom_bert_model")
#tokenizer.save_pretrained("./custom_bert_model")

Directory 'bert' created.


# Naive Bayes

## Data processing

In [27]:
df = pd.read_parquet("hf://datasets/google-research-datasets/go_emotions/raw/train-00000-of-00001.parquet")

In [28]:
labels = df.columns[9:]

In [29]:
df1 = df[df[labels].sum(axis=1) == 1]

In [30]:
comments = df1['text'].tolist()
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(comments)
y = df1[labels].values
y = np.argmax(y, axis=1)

## Naive Bayes Implementation

In [31]:
# Note X must be a sparse matrix. This is because the session crashes otherwise
# y is integer encoded not one hot encoded

class NaiveBayes:
    def __init__(self):
        self.px = None
        self.py = None

    def fit(self, X, y):
        # Shape of X is (N, L) where L is the length of the embedding vectors, X is a sparse matrix
        # Shape of y is (N,) where N is sample size

        self.X = X    # Bag of words representation
        self.y = y    # Integer labeled
        n = X.shape[0]
        C = len(np.unique(y))

        # Compute class priors (py)
        for c in range(C):
            self.py = np.bincount(y)[c] / n

        # Initialize px as a list to store likelihoods
        self.px = []

        for c in range(C):
            # Select samples where the class is c
            y_c = (y == c)  # Binary mask for samples with class c
            X_c = X[y_c == 1]  # Extract samples where class is c

            # Compute the likelihood P(x_i | y_c) for each feature
            px_c = (X_c.sum(axis=0) + 1) / (y_c.sum() + X.shape[1])
            px_c = np.asarray(px_c).ravel()  # Ensure it's a dense 1D array

            self.px.append(px_c)  # Add the likelihoods for class c

        # Convert px to numpy array of shape (C, L)
        self.px = np.array(self.px)

    def predict(self, X):
        # Compute the log of the posterior probabilities for each class
        log_py = np.log(self.py)  # Log of class priors
        log_px = np.log(self.px)  # Log of feature likelihoods

        # Compute log-posterior for each class (N samples, C classes)
        log_posterior = X.dot(log_px.T) + log_py  # `X` remains sparse

        # Return the class with the highest posterior probability for each sample
        return np.argmax(log_posterior, axis=1)

    def evaluate_acc(self, Y, Yh):
        return np.mean(Y == Yh)


In [32]:
model = NaiveBayes()
model.fit(X, y)
Yh = model.predict(X)
model.evaluate_acc(y, Yh)

0.41989873123035737

In [33]:
class_counts = np.bincount(y)
print("Class counts:", class_counts)


Class counts: [10531  6130  5202  8342 11259  3523  4938  5885  2147  4706  7686  2914
  1433  3020  1778  7075   351  4329  4957   796  4519   690  4714   788
  1510  3827  3472 55298]


In [34]:
baseline_accuracy = np.max(class_counts) / len(y)
print(f"Baseline Accuracy: {baseline_accuracy:.2f}")


Baseline Accuracy: 0.32
